# D2E2F: Helsingborg-Helsingør
(2021-04-23)

In [ ]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
plt.rcParams["figure.figsize"] = (20,3)
plt.style.use('presentation')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize
from src.data import get_dataset
from src.data import prepare_dataset
from src.data import trips
import scipy.integrate
import seaborn as sns
from ipywidgets import Layout, interact, IntSlider, interactive

In [ ]:
df = get_dataset.get(name='tycho_short_id')

## Plot trips

In [ ]:
visualize.plot_map(df=df, width=1400, height=800, zoom_start=14, color_key='trip_direction')

## Trip statistics

In [ ]:
df_stat = get_dataset.trip_statistics()

In [ ]:
df_stat_plot = df_stat.copy()

df_stat_plot['energy per trip [Nm]'] = df_stat_plot['power_em_thruster_total']

df_stat_plot['trip_direction'] = df_stat_plot['trip_direction'].apply(lambda x : 'Helsingør-Helsingborg' if x==0 else 'Helsingborg-Helsingør')

fig = px.scatter(df_stat_plot, x='start_time',y='energy per trip [Nm]', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'])
fig.show()

## Analyzing trips

In [ ]:
fig = px.line(df, x='trip_time', y='sog', color='trip_no', width=1500, height=800)
fig.show()

In [ ]:
def track_plot(time,x,y,psi,lpp,beam,ax,N=7,line_style = 'y',alpha = 1):

    indexes = np.linspace(0, len(time) - 1, N).astype(int)

    for i, index in enumerate(indexes):
        if i == 0:
            color = 'g'
            alpha_= 0.8
        elif i == (len(indexes) - 1):
            color = 'r'
            alpha_= 0.8
        else:
            color = line_style
            alpha_=0.5

        plot_ship(x[index], y[index], psi[index], lpp = lpp, beam = beam, ax=ax, color=color, alpha=alpha*alpha_)

def plot_ship(x,y,psi,ax,lpp,beam,color = 'y',alpha = 0.1):
    """Plot a simplified contour od this ship"""
    recalculated_boat = get_countour(x,y,psi,lpp = lpp, beam = beam)
    x = recalculated_boat[1]
    y = recalculated_boat[0]

    ax.plot(x,y,color,alpha=alpha)
    ax.fill(x, y, color, zorder=10,alpha=alpha)

def get_countour(x, y, psi,lpp,beam):
    # (Old Matlab boat.m)
    tt1 = lpp / 2
    tt2 = 0.9
    tt3 = beam / 4
    tt4 = 0.8
    tt5 = 3 * beam / 8
    tt6 = 0.6
    tt7 = beam / 2
    tt8 = 1.85 * beam / 4
    boat = np.matrix([[tt1, tt2 * tt1, tt4 * tt1, tt6 * tt1, -tt4 * tt1, -tt2 * tt1, -tt1, -tt1, -tt2 * tt1,
                       -tt4 * tt1, tt6 * tt1, tt4 * tt1, tt2 * tt1, tt1],
                      [0, -tt3, -tt5, -tt7, -tt7, -tt8, -tt5, tt5, tt8, tt7, tt7, tt5, tt3, 0]])
    delta = np.array([[x], [y]])

    rotation = np.matrix([[np.cos(psi), -np.sin(psi)],
                          [np.sin(psi), np.cos(psi)]])
    rows, columns = boat.shape
    rotated_boat = np.matrix(np.zeros((rows, columns)))
    for column in range(columns):
        rotated_boat[:, column] = rotation * boat[:, column]
    recalculated_boat = np.array(rotated_boat + delta)
    return recalculated_boat

In [ ]:
def plot_thruster(ax, x,y,cos,sin, power, scale=20):
    
    l = power*scale
    ax.arrow(x=x, y=y, dx=-l*sin, dy=-l*cos, head_width=l/5, head_length=l/5)
    
def plot_thrust_allocation(row, lpp=50, beam=20, scale=30):
    
    #fig,axes=plt.subplots(ncols=2)
    fig = plt.figure(constrained_layout=True)
    fig.set_size_inches(19,8)
    
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[:, 0])
    # identical to ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1:, 1])
    
    ax = ax1    
    positions = np.array([
        [-1,-1],
        [1,-1],
        [1,1],
        [-1,1],
        
    ])
    
    coordinates = np.array([positions[:,0]*beam/2, positions[:,1]*lpp/2] ).T

    if row['trip_direction']==1:
        # Rotate the view
        coordinates*=-1
    
    for i,position in enumerate(coordinates):
        
        n=i+1
        sin_key = 'sin_pm%i' % n
        cos_key = 'cos_pm%i' % n
        power_key = 'power_em_thruster_%i' % n
                
        plot_thruster(ax=ax, x=position[0], y=position[1], cos=row[cos_key], sin=row[sin_key], power=row[power_key], scale=scale)
    
    # velocity
    l=row['sog']
    direction = np.deg2rad(row['cog'] - row['heading'])
    dx = l*np.cos(direction)
    dy = l*np.sin(direction)
    ax.arrow(x=0, y=0, dx=dy, dy=dx, head_width=l/5, head_length=l/5, color='green')
    
    ax.set_xlim(np.min(coordinates[:,0])-scale,np.max(coordinates[:,0])+scale)
    ax.set_ylim(np.min(coordinates[:,1])-scale,np.max(coordinates[:,1])+scale)
    #ax.set_aspect('equal', 'box')
    
    ## Second plot:
    ax = ax2
    trip_.plot(x='longitude', y='latitude', ax=ax, style='k--')
    #ax.plot(row['longitude'], row['latitude'], 'o', ms=15)
    x = row['latitude']
    y = row['longitude']
    psi = np.deg2rad(row['cog'])
    
    plot_ship(x, y, psi, lpp = lpp_, beam = beam_, ax=ax, color='b', alpha=0.5)
    
    ax.set_ylim(trip_['latitude'].min(), trip_['latitude'].max())
    ax.set_xlim(trip_['longitude'].min(), trip_['longitude'].max())
    
    ax.set_aspect('equal')
    
    ## Third plot:
    trip_.plot(x='trip_time_s', y='sog', ax=ax3, style='k--')
    ax3.plot(row['trip_time_s'], row['sog'], 'o', ms=15)
    ax3.set_xlabel('Time [s]')
    ax3.set_ylabel('Ship speed [m/s]')
    
def animate(i=0):
    
    index = int(i)
    row = trip_.iloc[index]
    plot_thrust_allocation(row=row)

## Look at one trip

In [ ]:
trips = df.groupby(by='trip_no')
#trip = trips.get_group(110)
trip = trips.get_group(3)

trip_ = trip.copy()
trip_ = prepare_dataset.calculate_rudder_angles(df=trip_, drop=False)
trip_['trip_time_s'] = pd.TimedeltaIndex(trip_['trip_time']).total_seconds()

power_columns = ['power_em_thruster_%i' % i for i in range(1,5)]
trip_[power_columns]/=trip_['power_em_thruster_total'].max()/4

In [ ]:
visualize.plot_map(df=trip_, width=1400, height=800, zoom_start=14, color_key='trip_direction')

In [ ]:
N_scale = 20
lpp=50
beam=20
scale =  1/lpp/N_scale*np.sqrt((trip_['latitude'].max() - trip_['latitude'].min())**2 + (trip_['longitude'].max() - trip_['longitude'].min())**2)
lpp_ = lpp*scale
beam_ = beam*scale
N = 10

interact(animate, i = IntSlider(0,0,len(trip_)-1,1, layout=Layout(width='70%')));   

## Is there a more optimal thruster operation?

![](https://memegenerator.net/img/images/15921176.jpg)

### 1) Utveckla simuleringsmodell med Machine Learning
### 2) Använda simuleringsmodell för att optimera thrustrar 


### Analysarbete
#### a) Långtidsanalys: dagar/veckor/månader/år
#### b) Förbättring med "Smart Eco-shipping"
#### c) Korttidsanalys: farygsdynamik under resa
